# PantryPalML: Production Inference Notebook

This notebook demonstrates real inference using the saved production model and the `ProductionRecipeScorer` API.

It shows:
- Environment setup (Colab-friendly)
- Loading saved model + metadata
- Fetching user interactions with `UserInteractionFetcher`
- Generating top-N recommendations with `ProductionRecipeScorer`
- Basic validation and smoke tests


In [1]:
# Colab/Local environment setup (silent if local)
import sys, subprocess, os, pathlib

IN_COLAB = "google.colab" in sys.modules
repo_root = pathlib.Path.cwd()

if IN_COLAB:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                        "lightgbm", "pandas", "numpy", "scikit-learn"],
                       check=False)
    except Exception as e:
        print(f"pip install warning: {e}")

    if not (repo_root / "recipe_recommender").exists():
        subprocess.run(["git", "-c", "advice.detachedHead=false", "clone", "-q", "https://github.com/marcel-qayoom-taylor/PantryPalML.git"], check=True)
        os.chdir("PantryPalML")
        repo_root = pathlib.Path.cwd()

print(f"Environment ready. Project root: {repo_root}")


Environment ready. Project root: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/notebooks


In [ ]:
# Imports from production codebase
from pathlib import Path  # standard path utility

# Central paths + defaults (output/input/model, weights, params)
from recipe_recommender.config import get_ml_config

# Streams/filters combined_events.csv to user-level interaction list for recipes
from recipe_recommender.utils.fetch_user_interactions import UserInteractionFetcher

# Loads saved LightGBM + metadata, builds user profile features, scores all recipes
from recipe_recommender.inference.production_recipe_scorer import ProductionRecipeScorer

config = get_ml_config()
print("Model dir:", config.model_dir)
print("Output dir:", config.output_dir)


Model dir: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output/hybrid_models
Output dir: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output


### Load Model and Generate Recommendations
We load the saved LightGBM booster and metadata via `ProductionRecipeScorer` and score recipes for a user.


In [ ]:
# Fetch interactions and run inference
fetcher = UserInteractionFetcher(config)

# Example: pick a user with interactions (you can replace with a known user_id)
sample_user_id = None

# Quickly sample a user from events file
import pandas as pd
try:
    events_path = config.output_dir / "combined_events.csv"
    if events_path.exists():
        df_head = pd.read_csv(events_path, nrows=50000)
        # value_counts over recipe-related events finds an active user candidate
        candidates = (
            df_head[df_head["event"].isin(fetcher.recipe_events)]["distinct_id"].value_counts()
        )
        if len(candidates) > 0:
            sample_user_id = candidates.index[0]
except Exception as e:
    print("Warning reading events head:", e)

if sample_user_id is None:
    sample_user_id = "demo_user_123"  # fallback for demo

print("Using user:", sample_user_id)
# Returns list[dict] with event_type, timestamp, optional recipe_id
interactions = fetcher.fetch_user_interactions(sample_user_id)

# Loads saved booster + metadata, builds user profile features, scores all recipes
scorer = ProductionRecipeScorer(config)
# Returns ranked recommendations with scores and metadata
result = scorer.get_user_recipe_recommendations(sample_user_id, interactions, n_recommendations=10)

# Display top results
import pandas as pd
recs_df = pd.DataFrame(result["recommendations"])[:10]
recs_df[["recipe_id", "recipe_name", "score"]]


2025-09-20 16:28:54,858 - recipe_recommender.utils.fetch_user_interactions - INFO - 🔍 Initialized UserInteractionFetcher
2025-09-20 16:28:54,858 - recipe_recommender.utils.fetch_user_interactions - INFO -    Events file: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output/combined_events.csv
2025-09-20 16:28:54,859 - recipe_recommender.utils.fetch_user_interactions - INFO -    Tracking 8 event types
2025-09-20 16:28:54,946 - recipe_recommender.utils.fetch_user_interactions - INFO - 🎯 Fetching interactions for user: $device:E0E4C8E4-CC47-45B3-B283-3C46D8636904
2025-09-20 16:28:54,947 - recipe_recommender.utils.fetch_user_interactions - INFO - 📖 Reading events file in chunks...
2025-09-20 16:28:55,048 - recipe_recommender.utils.fetch_user_interactions - INFO -    Processed 50,000 rows, found 381 matches...


Using user: $device:E0E4C8E4-CC47-45B3-B283-3C46D8636904


2025-09-20 16:28:55,084 - recipe_recommender.utils.fetch_user_interactions - INFO - ✅ Found 381 recipe interactions for user $device:E0E4C8E4-CC47-45B3-B283-3C46D8636904
2025-09-20 16:28:55,084 - recipe_recommender.utils.fetch_user_interactions - INFO -    Processed 63,662 total events
2025-09-20 16:28:55,084 - recipe_recommender.utils.fetch_user_interactions - INFO -    Time range: 2024-05-25 to 2024-10-01
2025-09-20 16:28:55,085 - recipe_recommender.utils.fetch_user_interactions - INFO -    Event breakdown: {'Recipe Favourited': 10, 'Recipe Viewed': 349, 'Recipe Link Clicked': 22}
2025-09-20 16:28:55,085 - recipe_recommender.inference.production_recipe_scorer - INFO - 🎯 Initializing Production Recipe Scorer...
2025-09-20 16:28:55,086 - recipe_recommender.inference.production_recipe_scorer - INFO - 🔧 Loading trained model components...
2025-09-20 16:28:55,092 - recipe_recommender.inference.production_recipe_scorer - INFO - ✅ Loaded LIGHTGBM model
2025-09-20 16:28:55,093 - recipe_recom

,recipe_id,recipe_name,score,servings,total_time
0,1081,Thai Coconut Chicken,0.130396,5,25
1,255,Chinese Lettuce Wraps (San Choy Bow ),0.123584,3,20
2,259,Chinese Satay Chicken Stir Fry,0.123584,2,20
3,488,Glass noodle salad with lime cashew crumble,0.123584,3,20
4,938,Satay Chicken Noodle Salad,0.123584,3,20
5,160,Celebration Salmon Salad,0.117588,5,35
6,1036,Sticky Chicken With Vegetable Fried Rice,0.115913,2,20
7,1043,Stir Fried Peanut Sauce Noodles,0.112112,2,23
8,206,Chicken Nachos,0.109643,4,25
9,1133,Vegetarian lettuce wraps,0.109643,2,25


In [4]:
# Smoke tests
errors = []

# 1) Model artifacts exist
from pathlib import Path
model_file = config.model_dir / "hybrid_lightgbm_model.txt"
meta_file = config.model_dir / "hybrid_lightgbm_metadata.json"
if not model_file.exists():
    errors.append(f"Missing model file: {model_file}")
if not meta_file.exists():
    errors.append(f"Missing metadata file: {meta_file}")

# 2) Recipe feature file exists
recipe_file = config.output_dir / "enhanced_recipe_features_from_db.csv"
if not recipe_file.exists():
    errors.append(f"Missing recipe features file: {recipe_file}")

# 3) Recommendation output sanity
if len(result.get("recommendations", [])) == 0:
    errors.append("No recommendations returned")

if errors:
    print("SMOKE TEST: FAIL")
    for e in errors:
        print(" - ", e)
else:
    print("SMOKE TEST: PASS")
    print(f"Top recommendation: {recs_df.iloc[0]['recipe_name']} (score={recs_df.iloc[0]['score']:.4f})")


SMOKE TEST: PASS
Top recommendation: Thai Coconut Chicken (score=0.1304)
